In [49]:
%matplotlib inline
from pandas import DataFrame as df
from scipy import ndimage as ndi
import matplotlib.pyplot as plt
import skimage
from skimage import data, img_as_float,color, img_as_ubyte
from skimage import color, io
from skimage.feature import peak_local_max, canny
from skimage.transform import hough_circle,hough_circle_peaks,hough_line, hough_line_peaks
from skimage.draw import circle_perimeter
from skimage.filters import try_all_threshold, threshold_otsu, threshold_triangle
from skimage.morphology import watershed, skeletonize
from skimage.segmentation import chan_vese, felzenszwalb, slic, quickshift, watershed,mark_boundaries
from skimage.measure import regionprops
from matplotlib import cm
import numpy as np

In [50]:
loc_l = 'images/01/'
loc_u = '.tif'
centroid_x = []
centroid_y = []
object_id = []
Timepoint = []
for i in range(1,93):
    loc = loc_l + str(i) + loc_u
    img = io.imread(loc)
    img = img_as_float(img)
    thresh = skimage.filters.threshold_mean(img)
    binary = img > thresh
    distance = ndi.distance_transform_edt(binary)
    local_maxi = np.copy(distance)
    local_maxi = local_maxi>(np.max(local_maxi)*0.1)
    coordinates = np.where(local_maxi)
    markers = ndi.label(local_maxi)[0]
    labels = watershed(-distance, markers, mask=binary)
    props = regionprops(labels)
    for prop in props:
        Timepoint.append(i-1)
        y0,x0 = prop.centroid
        centroid_x.append(x0)
        centroid_y.append(y0)
object_id = [i for i in range(1,len(centroid_x)+1)]
Track_id = [0 for i in range(1,len(centroid_x)+1)]

In [51]:
total_data = [object_id,centroid_x,centroid_y,Timepoint,Track_id]

In [52]:
data = df(object_id,columns = ['object_ID'])
data['Centroid_X'] = centroid_x
data['Centroid_Y'] = centroid_y
data['TimePoint'] = Timepoint
data['Track_ID'] = Track_id
data

,object_ID,Centroid_X,Centroid_Y,TimePoint,Track_ID
0,1,568.976240,43.481749,0,0
1,2,193.492887,104.489989,0,0
2,3,173.250000,84.250000,0,0
3,4,185.875000,114.291667,0,0
4,5,190.947368,128.315789,0,0
...,...,...,...,...,...
4672,4673,721.513725,960.623529,91,0
4673,4674,738.585859,959.868687,91,0
4674,4675,725.967089,975.088608,91,0
4675,4676,750.706767,971.355890,91,0


In [53]:
import csv
import io

In [55]:
data.to_csv("Track.csv")

'filename = "Tracks.csv"\nwith io.open(filename, \'w\', encoding="utf-8") as csvfile:  \n    csvwriter = csv.writer(csvfile)  \n    csvwriter.writerows(data)'